In [1]:
!git clone https://github.com/WongKinYiu/yolov9

fatal: destination path 'yolov9' already exists and is not an empty directory.


In [10]:
!cd yolov9

In [1]:
!pip install -r requirements.txt

In [2]:
!pip install -r setup.txt

In [2]:
import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from models.common import DetectMultiBackend, AutoShape


In [13]:
# Khởi tạo YOLOv9
device = "cuda"
device = torch.device(device)
model  = DetectMultiBackend(weights="weights/yolov9-s-converted.pt", device=device, fuse=True )
model  = AutoShape(model)

# Load classname từ file classes.names
with open("data_ext/classes.names") as f:
    class_names = f.read().strip().split('\n')
print(class_names)
colors = np.random.randint(0,255, size=(len(class_names),3 ))




d:\Project\People_Tracking\yolov9_deepsort\yolov9\models\experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_download(w), map_locati

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'potted plant', 'bed', 'dining table', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [11]:

# Config value
conf_threshold = 0.5
tracking_class = 0
video_path = "data_ext/walking_1.mp4"

# Khởi tạo VideoCapture để đọc từ file video
cap = cv2.VideoCapture(video_path)


# Lấy thông tin về kích thước và FPS của video gốc
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Khởi tạo VideoWriter để lưu video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho MP4
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

# Khởi tạo DeepSort
tracker = DeepSort(max_age=30)
tracks = []

# Tiến hành đọc từng frame từ video
while True:
    # Đọc
    ret, frame = cap.read()
    if not ret:
      break

    # Đưa qua model để detect
    results = model(frame)
    print(results)
    detect = []
    for detect_object in results.pred[0]:
        label, confidence, bbox = detect_object[5], detect_object[4], detect_object[:4]
        x1, y1, x2, y2 = map(int, bbox)
        class_id = int(label)
        print(label, confidence, bbox, class_id, label)
        if tracking_class is None:
            if confidence < conf_threshold:
                continue
        else:
            if class_id != tracking_class or confidence < conf_threshold:
                continue
        detect.append([[x1, y1, x2 - x1, y2 - y1], confidence, class_id])

    print(detect)
    # Cập nhật, gán ID bằng DeepSort
    tracks = tracker.update_tracks(detect, frame=frame)

    # Vẽ lên màn hình các khung chữ nhật kèm ID
    for track in tracks:
        if track.is_confirmed():
            track_id = track.track_id
            print(track)
            # Lấy tọa độ, class_id để vẽ lên hình ảnh
            ltrb = track.to_ltrb()
            class_id = track.get_det_class()
            x1, y1, x2, y2 = map(int, ltrb)
            color = colors[class_id]
            B, G, R = map(int, color)

            label = "{}-{}".format(class_names[class_id], track_id)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
            cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(label) * 12, y1), (B, G, R), -1)
            cv2.putText(frame, label, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Ghi frame vào file video output
    out.write(frame)


# Giải phóng tài nguyên
cap.release()
out.release()


image 1/1: 1080x1920 36 persons, 1 tennis racket
Speed: 4.0ms pre-process, 89.5ms inference, 2.0ms NMS per image at shape (1, 3, 384, 640)
tensor(0., device='cuda:0') tensor(0.87000, device='cuda:0') tensor([1140.93848,  950.59662, 1244.26208, 1079.82324], device='cuda:0') 0 tensor(0., device='cuda:0')
tensor(0., device='cuda:0') tensor(0.86957, device='cuda:0') tensor([665.39850, 650.42151, 744.13574, 852.25989], device='cuda:0') 0 tensor(0., device='cuda:0')
tensor(0., device='cuda:0') tensor(0.84096, device='cuda:0') tensor([259.26740, 440.47778, 329.55774, 610.80615], device='cuda:0') 0 tensor(0., device='cuda:0')
tensor(0., device='cuda:0') tensor(0.83787, device='cuda:0') tensor([1445.47070,  319.73923, 1515.94580,  484.54514], device='cuda:0') 0 tensor(0., device='cuda:0')
tensor(0., device='cuda:0') tensor(0.83421, device='cuda:0') tensor([749.56323, 451.81879, 822.33856, 622.34619], device='cuda:0') 0 tensor(0., device='cuda:0')
tensor(0., device='cuda:0') tensor(0.83007, devi

### Real-time tracking

In [16]:
import cv2
import time

# Initialize VideoCapture to read from a video source
cap = cv2.VideoCapture(1)

# Initialize DeepSort
tracker = DeepSort(max_age=30)
tracks = []

# Variables to calculate FPS
prev_time = 0

# Process each frame
while True:
    # Read a frame
    ret, frame = cap.read()
    if not ret:
        break

    # Get the current time
    current_time = time.time()
    
    # Calculate FPS
    fps = 1 / (current_time - prev_time) if prev_time else 0
    prev_time = current_time

    # Pass the frame through the model for detection (replace `model` with your actual model)
    results = model(frame)
    detect = []
    for detect_object in results.pred[0]:
        label, confidence, bbox = detect_object[5], detect_object[4], detect_object[:4]
        x1, y1, x2, y2 = map(int, bbox)
        class_id = int(label)
        
        if confidence < conf_threshold:
            continue
        detect.append([[x1, y1, x2 - x1, y2 - y1], confidence, class_id])

    # Update tracks using DeepSort
    tracks = tracker.update_tracks(detect, frame=frame)

    # Draw bounding boxes and labels
    for track in tracks:
        if track.is_confirmed():
            track_id = track.track_id
            ltrb = track.to_ltrb()
            class_id = track.get_det_class()
            x1, y1, x2, y2 = map(int, ltrb)
            color = (255, 0, 0)  # Set your color logic
            label = f"ID-{track_id}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display FPS on the frame
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
